In [78]:
import numpy as np
class board:
    def __init__(self,m=18,n=9):
        self.m=m
        self.n=n
        self.pg=[]
        self.obsticles=[]
    def add_obsticle(self,obsticles):
        self.obsticles=obsticles
        self.dots=np.ones([self.m,self.n])
    def add_pg(self,pg):
        if (pg.x,pg.y) not in self.obsticles :
            self.pg.append(pg)
            return True
        return False
    def request_move(self,w,index):
        pg=self.pg[index]
        if w==1  and pg.y<self.n-1:
            next=np.array([pg.x,pg.y+1])
        elif w==2  and pg.x<self.m-1:
            next=np.array([pg.x+1,pg.y])
        elif w==3  and pg.y>0:
            next=np.array([pg.x,pg.y-1])
        elif w==4 and pg.x>0 :
            next=np.array([pg.x-1,pg.y])
        else:
            next=np.array([pg.x,pg.y])
        if index==0 :
            pg.score=pg.score-1
            if self.dots[next[0],next[1]]==1:
                self.dots[next[0],next[1]]=0
                pg.score=pg.score+10
        if not((self.obsticles==next).all(axis=1)).any():
            pg.x,pg.y=next[0],next[1]
            return True
        return False
    def Map(self):
        b=np.zeros([self.m,self.n],dtype=np.int8)
        for i in self.pg:
            b[i.x,i.y]=2
        b[self.pg[0].x,self.pg[0].y]=1
        b[self.obsticles[:,0],self.obsticles[:,1]]=-1
        return b  
    def step(self):
        for i in self.pg:
             i.go()          
        

In [33]:
class PG:
    def __init__(self,x,y,enviroment=None):
        self.x=x
        self.y=y
        self.enviroment=enviroment
        self.score=10
        if not enviroment==None:
            enviroment.add_pg(self)
            self.index=len(enviroment.pg)-1
    def move(self,w):
        if w not in [1,2,3,4]:
            return False
        elif self.enviroment.request_move(w,self.index):
                self.x,self.y=self.enviroment.pg[self.index].x,self.enviroment.pg[self.index].y


In [34]:
class Pacman(PG):
    def __init__(self, x, y, enviroment=None):
        super().__init__(x, y, enviroment)
        if not enviroment==None:
            self.enviroment.dots[x,y]=0
    def go(self):
        self.move(np.random.choice(4)+1)

In [35]:
class ghost(PG):
    def go(self):
        self.move(np.random.choice(4)+1)


In [79]:
m=18
n=9
b=board(m,n)
b.add_obsticle(obs)
p=Pacman(4,2,b)
g1=ghost(7,8,b)
g2=ghost(5,5,b)
# obs=np.random.random_integers(18)
obs=np.array([[1,2],[3,4]])
# np.random.choice(np.arange(10,20), size=(5,2), replace=True)


In [80]:
b.step()
b.Map()

self.dots[7,5][[1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]] , next: [7 8],self.dots[next]: [[1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]]


array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  2,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int8)

In [77]:
a=np.ones([10,10])
b=np.array([5,1,2])
a[1,2]

1.0